In [1]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules, apriori

# Read dataset

In [17]:
sales_receipts = pd.read_csv("dataset/201904 sales reciepts.csv")

In [18]:
products = pd.read_csv("dataset/product.csv")

# Data merge

In [19]:
sales_receipts = sales_receipts[["transaction_id", "transaction_date", "customer_id", "product_id", "sales_outlet_id", "quantity"]]
products = products[["product_id", "product", "product_category"]]

In [20]:
dataset = pd.merge(sales_receipts, products, on="product_id", how="left")

In [21]:
dataset.head()

,transaction_id,transaction_date,customer_id,product_id,sales_outlet_id,quantity,product,product_category
0,7,2019-04-01,558,52,3,1,Traditional Blend Chai Rg,Tea
1,11,2019-04-01,781,27,3,2,Brazilian Lg,Coffee
2,19,2019-04-01,788,46,3,2,Serenity Green Tea Rg,Tea
3,32,2019-04-01,683,23,3,2,Our Old Time Diner Blend Rg,Coffee
4,33,2019-04-01,99,34,3,1,Jamaican Coffee River Sm,Coffee


## Remove sizes

In [22]:
dataset[dataset["product"].str.contains("Dark chocolate")]["product"].unique()

array(['Dark chocolate Lg', 'Dark chocolate Rg', 'Dark chocolate'],
      dtype=object)

In [23]:
dataset["product"].nunique()

80

In [24]:
dataset["product"] = dataset["product"].str.replace(' Rg', '')
dataset["product"] = dataset["product"].str.replace(' Lg', '')
dataset["product"] = dataset["product"].str.replace(' Sm', '')


In [25]:
dataset.head()
dataset["product"].nunique()

45

In [ ]:
print(dataset["product"].unique())

## Choose product subset

In [27]:
products_to_keep = ["Cappuccino", "Latte", "Espresso shot", "Dark chocolate", "Sugar Free Vanilla syrup", "Carmel syrup",
                    "Chocolate syrup", "Hazelnut syrup", "Ginger Scone", "Chocolate Croissant", "Jumbo Savory Scone",
                    "Cranberry Scone", "Hazelnut Biscotti", "Croissant", "Almond Croissant", "Oatmeal Scone", "Chocolate Chip Biscotti",
                    "Ginger Biscotti"
                    ]

In [34]:
dataset = dataset[dataset["product"].isin(products_to_keep)]

In [35]:
dataset.head()

,transaction_id,transaction_date,customer_id,product_id,sales_outlet_id,quantity,product,product_category,transaction
16,108,2019-04-01,65,40,3,1,Cappuccino,Coffee,108_65
17,112,2019-04-01,90,37,3,2,Espresso shot,Coffee,112_90
20,127,2019-04-01,116,41,3,2,Cappuccino,Coffee,127_116
21,134,2019-04-01,189,38,3,2,Latte,Coffee,134_189
22,135,2019-04-01,131,40,3,1,Cappuccino,Coffee,135_131


## Clean transactions

In [ ]:
dataset["transaction"] = dataset["transaction_id"].astype(str) + "_" + dataset["customer_id"].astype(str)

In [37]:
num_of_items_per_transaction = dataset["transaction"].value_counts().reset_index()
num_of_items_per_transaction

,transaction,count
0,209_0,31
1,206_0,30
2,204_0,27
3,208_0,25
4,203_0,24
...,...,...
8381,135_523,1
8382,130_157,1
8383,121_465,1
8384,118_748,1


In [38]:
valid_transaction = num_of_items_per_transaction[(num_of_items_per_transaction["count"] > 1)]["transaction"].tolist()

In [39]:
dataset = dataset[dataset["transaction"].isin(valid_transaction)]

In [40]:
dataset.shape

(10189, 9)

In [41]:
dataset["product"].value_counts()
dataset["product_category"].value_counts()

product_category
Bakery                3800
Coffee                3174
Flavours              2246
Drinking Chocolate     947
Packaged Chocolate      22
Name: count, dtype: int64

# Popularity recommendation engine

In [42]:
product_recommendation = dataset.groupby(["product", "product_category"]).count().reset_index()

In [43]:
product_recommendation.head()

,product,product_category,transaction_id,transaction_date,customer_id,product_id,sales_outlet_id,quantity,transaction
0,Almond Croissant,Bakery,347,347,347,347,347,347,347
1,Cappuccino,Coffee,1290,1290,1290,1290,1290,1290,1290
2,Carmel syrup,Flavours,561,561,561,561,561,561,561
3,Chocolate Chip Biscotti,Bakery,352,352,352,352,352,352,352
4,Chocolate Croissant,Bakery,636,636,636,636,636,636,636


In [44]:
product_recommendation = product_recommendation[["product", "product_category", "transaction_id"]]
product_recommendation = product_recommendation.rename(columns={"transaction_id": "num_of_transactions"})

In [46]:
product_recommendation.to_csv("api/recommendation_dataset/popularity_recommendation.csv", index=False)

# Apriori recommendation engine

In [47]:
train_basket = (dataset.groupby(["transaction", "product"])["product"].count().reset_index(name="count"))

In [50]:
wide_basket = train_basket.pivot_table(index="transaction", columns="product", values="count").fillna(0)
wide_basket.head()

In [53]:
def encode_units(x):
    if x <= 0:
        return 0
    else:
        return 1

wide_basket = wide_basket.map(encode_units)

In [ ]:
wide_basket.head()

In [55]:
frequent_items = apriori(wide_basket, min_support=0.05, use_colnames=True)

In [ ]:
frequent_items.head(20)

In [59]:
rules_basket = association_rules(frequent_items, metric="lift", min_threshold=1)

In [60]:
rules_basket.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Cappuccino),(Almond Croissant),0.388889,0.115646,0.053288,0.137026,1.184874,0.008314,1.024775,0.255319
1,(Almond Croissant),(Cappuccino),0.115646,0.388889,0.053288,0.460784,1.184874,0.008314,1.133333,0.176432
2,(Almond Croissant),(Dark chocolate),0.115646,0.277022,0.057445,0.496732,1.793115,0.025409,1.436567,0.500152
3,(Dark chocolate),(Almond Croissant),0.277022,0.115646,0.057445,0.207367,1.793115,0.025409,1.115717,0.611791
4,(Almond Croissant),(Latte),0.115646,0.382086,0.054422,0.470588,1.231629,0.010235,1.167171,0.212660


In [ ]:
rules_basket[rules_basket["antecedents"] == {"Latte"}].sort_values("confidence", ascending=False)

## Save in json format

In [62]:
product_categories = dataset[["product", "product_category"]].drop_duplicates().set_index("product").to_dict()["product_category"]

In [63]:
recommendations_json = {}
antecedents = rules_basket["antecedents"].unique()
for antecedent in antecedents:
    df_rec = rules_basket[rules_basket["antecedents"] == antecedent]
    df_rec = df_rec.sort_values("confidence", ascending=False)
    key = "_".join(antecedent)
    recommendations_json[key] = []
    for _,row in df_rec.iterrows():
        objects = row["consequents"]
        for object in objects:
            already_exist = False
            for current_object in recommendations_json[key]:
                if object == current_object["product"]:
                    already_exist = True
            if already_exist:
                continue
            rec_dict = {
                "product": object,
                "product_category": product_categories[object],
                "confidence": row["confidence"]
            }
            recommendations_json[key].append(rec_dict)



In [64]:
recommendations_json

{'Cappuccino': [{'product': 'Sugar Free Vanilla syrup',
   'product_category': 'Flavours',
   'confidence': 0.29057337220602525},
  {'product': 'Chocolate syrup',
   'product_category': 'Flavours',
   'confidence': 0.2818270165208941},
  {'product': 'Carmel syrup',
   'product_category': 'Flavours',
   'confidence': 0.26433430515063167},
  {'product': 'Hazelnut syrup',
   'product_category': 'Flavours',
   'confidence': 0.23226433430515062},
  {'product': 'Ginger Scone',
   'product_category': 'Bakery',
   'confidence': 0.15354713313896987},
  {'product': 'Chocolate Chip Biscotti',
   'product_category': 'Bakery',
   'confidence': 0.14480077745383868},
  {'product': 'Chocolate Croissant',
   'product_category': 'Bakery',
   'confidence': 0.14480077745383868},
  {'product': 'Croissant',
   'product_category': 'Bakery',
   'confidence': 0.14480077745383868},
  {'product': 'Jumbo Savory Scone',
   'product_category': 'Bakery',
   'confidence': 0.14188532555879496},
  {'product': 'Ginger B